#### Download Files from gdc

In [ ]:
import os
import sys

manifest = input("Enter the manifest file name: ")

if not os.path.exists(manifest):
    print("File not found")
if not os.path.exists("gdc_download"):
    os.makedirs("gdc_download")
os.system("gdc-client download -m " + manifest + " -d gdc_download")


In [ ]:
import os
import shutil

def move_idat_files_to_parent(target_directory):
    if not os.path.isdir(target_directory):
        print(f"{target_directory} is not a valid directory.")
        return

    for subdir in os.listdir(target_directory):
        subdir_path = os.path.join(target_directory, subdir)

        if os.path.isdir(subdir_path):
            for file in os.listdir(subdir_path):
                if file.endswith(".idat"):
                    file_path = os.path.join(subdir_path, file)
                    dest_path = os.path.join(target_directory, file)
                    try:
                        shutil.move(file_path, dest_path)
                        print(f"Moved: {file_path} -> {dest_path}")
                    except Exception as e:
                        print(f"Error moving {file_path}: {e}")


base_directory = input("dgc download directory: ")
if base_directory == "":
    base_directory = "gdc_download"
move_idat_files_to_parent(base_directory)


In [ ]:
import os
import shutil

def move_idat_files(source_dir, target_dir):
    if not os.path.isdir(source_dir):
        print(f"{source_dir} is not a valid directory.")
        return

    if not os.path.exists(target_dir):
        try:
            os.makedirs(target_dir)
            print(f"Created target directory: {target_dir}")
        except Exception as e:
            print(f"Error creating target directory {target_dir}: {e}")
            return

    for subdir in os.listdir(source_dir):
        subdir_path = os.path.join(source_dir, subdir)

        if os.path.isdir(subdir_path):
            for file in os.listdir(subdir_path):
                if file.endswith(".idat"):
                    file_path = os.path.join(subdir_path, file)
                    dest_path = os.path.join(target_dir, file)

                    try:
                        shutil.move(file_path, dest_path)
                        print(f"Moved: {file_path} -> {dest_path}")
                    except Exception as e:
                        print(f"Error moving {file_path}: {e}")


source_directory = input("Enter the path to the source directory: ")
if source_directory == "":
    source_directory = "gdc_download"
target_directory = input("Enter the path to the target directory: ")
if target_directory == "":
    target_directory = "idats"
move_idat_files(source_directory, target_directory)


#### Prapare sample sheet


In [ ]:
import pandas as pd

df = pd.read_csv(input("Enter the name of sample sheet from gdc: "), sep="\t")

In [ ]:
df_sorted = df.sort_values(by='Sample Type')
df_sorted = df_sorted[["Sample Type", "File Name"]]
df_normal = df_sorted[df_sorted["Sample Type"] == "Solid Tissue Normal"]
df_tumor = df_sorted[df_sorted["Sample Type"] == "Primary Tumor"]
df_normal = df_normal.sort_values(by='File Name')
df_tumor = df_tumor.sort_values(by='File Name')

In [ ]:
def split_string_by_underline(string):
    return string.split('_')

In [ ]:
df_concat = pd.concat([df_normal, df_tumor], axis=0)
df_concat.reset_index(drop=True, inplace=True)
df_concat["Sample_Name"] = df_concat.index + 1
df_concat["Sample_Plate"] = ""
df_concat["Pool_ID"] = ""
df_concat.rename(columns={"Sample Type": "Sample_Group"}, inplace=True)
df_concat["Sentrix_ID"] = df_concat["File Name"].apply(lambda x: split_string_by_underline(x)[0])
df_concat["Sentrix_Position"] = df_concat["File Name"].apply(lambda x: split_string_by_underline(x)[1])
df_concat["Sample_Well"] = ""
df_concat.drop(columns=["File Name"], inplace=True)

In [ ]:
df_concat = df_concat[["Sample_Name", "Sample_Plate", "Sample_Group", "Pool_ID", "Sentrix_ID", "Sentrix_Position", "Sample_Well"]]
df_concat['Sample_Group'] = df_concat['Sample_Group'].str.replace(' ', '_')
df_concat

In [ ]:
df_concat.to_csv('sample_sheet_BRCA.csv', index=False)

#### Check idats

In [ ]:
import pandas as pd
df_name = input("Enter the name of sample sheet for champ")
df = pd.read_csv(df_name)
df

In [ ]:
import os
import shutil

def check(folder):
    if not os.path.isdir(folder):
        print(f"{folder} is not a valid directory.")
        return
    idat_files = [f for f in os.listdir(folder) if f.endswith('.idat')]
    sample_ids = [f.split('_')[0] for f in idat_files]
    flag = True
    # Check if each sample ID exists in the dataframe
    for sample_id in sample_ids:
        if sample_id not in df['Sentrix_ID'].values:
            flag = False
            print(f"Warning: Sample {sample_id} not found in sample sheet")
        else:
            pass
    if flag:
        print("All samples found in sample sheet")
# F:\Repo\btc\data\TCGA_COAD_READ
# gdc_sample_sheet.2024-12-05.modified.csv
folder = input("Enter the path to the folder for checking")

check(folder)
